# Benchmarking Our Algorithm against others

In [95]:
import sys
sys.path.append('../')
from panav.environment.env import Room
from panav.hybrid import HybridGraph
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:

bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
n_agents = list(range(10,101,10))
bloating_r = 0.5
vmax = 1.0



TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }
env_name = "Room"



In [97]:
RESUME = False

In [98]:
import os
from panav.hybrid import reduced_agents_HG
from panav.checkpoint import get_latest_checkpoint
import signal

def timeout_handler(signum,frame):
    print("Algorithm timeout")
    raise TimeoutError("Timeout")

N_idx = 0
alg_idx = 0
results = []
timeouted = {alg:False for alg in algs}
iteration = 0

def save_checkpoint(checkpoint_dir,env,HG_main,iteration):
    checkpoint = {
        'N_idx':N_idx,
        'alg_idx':alg_idx,
        'results':results,
        'timeouted':timeouted,
        'env':env,
        'HG_main':HG_main,
        'iteration':iteration
    }

    with open(checkpoint_dir+f"_{iteration}_.ckpt",'wb') as fp:
        pkl.dump(checkpoint,fp)


import shutil
checkpoint_dir = f"./data/checkpoints/{env_name}/"
if not RESUME:
    if env_name in os.listdir("./data/checkpoints"):
        shutil.rmtree(checkpoint_dir)
    os.mkdir(checkpoint_dir)
    

    env = Room(N_agent=max(n_agents))
    print('Constructing environment')
    HG_main = HybridGraph(env,agent_radius=bloating_r,tunnels=env.tunnels)
    save_checkpoint(checkpoint_dir,env,HG_main,0)
else:
    latest_checkpoint = get_latest_checkpoint(checkpoint_dir)
    with open(checkpoint_dir+latest_checkpoint,'rb') as fp:
        checkpoint = pkl.load(fp)
    N_idx = checkpoint['N_idx']
    alg_idx = checkpoint['alg_idx']
    results = checkpoint['results']
    timeouted = checkpoint['timeouted']
    env = checkpoint['env']
    HG_main = checkpoint['HG_main']
    iteration = checkpoint['iteration']


names = list(algs.keys())

while N_idx<len(n_agents):
    N = n_agents[N_idx]
    HG = reduced_agents_HG(HG_main,N)    
    while True:            
        name = names[alg_idx]
        alg = algs[name]

        if not timeouted[name]:
            
            save_checkpoint(checkpoint_dir,env,HG_main,iteration)
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump(results,fp)
        
            print("Alg:", name, "N agent:",N)

            result = {}
            

            t0 = time()

            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                plan = alg(HG)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['plan'] = plan
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["HG"]=HG

            results.append(result)

            
            if t>TIMEOUT:
                timeouted[name] = True
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                print("On time for algorithm", name, "N agent = ", N)

        alg_idx += 1
        iteration += 1

        if alg_idx == len(algs):
            alg_idx = 0
            break
    N_idx += 1


Alg: TAHP N agent: 100


KeyboardInterrupt: 